In [1]:
import sqlite3

In [22]:
db = sqlite3.connect('data_copy.sqlite3')

In [24]:
with db as conn:
    conn.execute('ALTER TABLE corpus_object ADD COLUMN path')

OperationalError: duplicate column name: path

In [14]:
path_subquery = lambda colname: f'''(
    WITH RECURSIVE parent_of(oid, parent, distance, path) AS (
        SELECT frob.oid, frob.parent, 0, NULL FROM corpus_object frob WHERE frob.oid={colname}
        UNION ALL
        SELECT parent_of.oid, narf.parent, parent_of.distance+1,
            CASE WHEN parent_of.path NOTNULL THEN narf.name || "/" || parent_of.path ELSE narf.name END
        FROM corpus_object narf JOIN parent_of ON narf.oid = parent_of.parent
    )
    SELECT path FROM parent_of ORDER BY distance DESC LIMIT 1)'''

In [25]:
with db as conn:
    conn.execute(f'UPDATE corpus_object SET path = {path_subquery("corpus_object.oid")}')

In [27]:
with db as conn:
    data = conn.execute('SELECT * FROM corpus_object LIMIT 10 OFFSET 10000').fetchall()
data

[(10002,
  '25SQWTCG5RDFPKKDZP5PLVUZDU',
  'Personenbeischriften (links)',
  7,
  16321,
  '{}',
  'aaew/bbawgrabinschriften/〈Grabinschriften des Alten Reiches〉/Sakkara/Unas-Friedhof/Mastaba des Nianch-Chnum und Chnum-hotep/Felskammer/Ostwand/Szene 30 (Südteil)'),
 (10003,
  '25VK4X56V5DXBNC2WGJ223KULA',
  'Westseite',
  2,
  18677,
  '{"definition": {"main_group": {"definition": "PM III 284 Outer Room", "protocol": "erfa\\u00dft August 2004"}}, "bibliography": {"bibliographical_text_field": "S. Hassan, Excavations at G\\u00eeza 1929-1930, Oxford 1932, 78-79, fig. 138, pl. XLIX.2 [*F, K, H, *P]"}, "object": {"description_of_object": {"type": "HV3JFDDE4NDJ5HTBR7F2QWPHFE", "agent": "GT4HRUBG55AE5IB7RM2ZMJOQ6M"}}, "find_spot": {"find_spot": {"place": {"place": "TZ7HFSUO3ZCLLAAXX3AEWXALIE"}}}, "text": {"textual_metadata": {"script": "TYXKD55ZDRB5NAWUTC7YWWJQRQ", "texttype": "S6TT23BYFJAWBEURQQIZYVRFXI"}}, "date": {"date": {"date": "GH2RXLTQ2BCEHPQ6R2SPBER7OM"}}}',
  'aaew/bbawgrabinschrift

In [28]:
db.close()